In [ ]:
import sys
sys.path.append('../src') 
from political_ads.preprocessor import Preprocessor

preprocess = Preprocessor()
data = preprocess.file_to_df("..\\data\\all_politicians_aggregated_final.txt")

In [ ]:
from political_ads.keyword_filter import Filter

# filter object
filter = Filter()

climate_ads = filter.get_climate_ads(data) 

### Preprocess

In [ ]:
#install the package paramiko
!pip install paramiko
!pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np

In [ ]:
def lemmatize_stemming(text):
    stemmer = SnowballStemmer('english')
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
preprocessed_climate_ads = climate_ads["ad_creative_body"].map(preprocess)

### Bag of words on data set
Create a dictionary from ‘processed_docs’ containing the number of times a word appears in the training set.

In [ ]:
# Will index each word in the corpus with a unique integer.
# Contains words counts 
dictionary = gensim.corpora.Dictionary(preprocessed_climate_ads)

In [ ]:
#Filter out tokens that appear in
#less than 15 documents (absolute number) or
#more than 0.5 documents (fraction of total corpus size, not absolute number).
#after the above two steps, keep only the first 100000 most frequent tokens.

dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
print(len(dictionary.iteritems()))

In [ ]:
#For each document we create a dictionary reporting how many
#words and how many times those words appear. Save this to ‘bow_corpus’, then check our selected document earlier.
bow_corpus = [dictionary.doc2bow(doc) for doc in preprocessed_climate_ads]

# Preview
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

### Run LDA model using bag of words

In [40]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, 
                                       id2word=dictionary, passes=2, 
                                       workers=2)

C:\Users\laise\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Visualization

In [ ]:
!pip install pyLDAvis
!pip install pyLDAvis.gensim

In [41]:
%matplotlib inline
import pyLDAvis
import pyLDAvis.gensim_models
import gensim

vis = pyLDAvis.gensim_models.prepare(lda_model, bow_corpus, dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

C:\Users\laise\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Allocating topics to documents

In [19]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.6108837127685547	 
Topic: 0.030*"fuel" + 0.029*"green" + 0.026*"deal" + 0.026*"fossil" + 0.018*"crisi" + 0.018*"action" + 0.018*"fight" + 0.013*"industri" + 0.013*"support" + 0.013*"movement"

Score: 0.3391064703464508	 
Topic: 0.030*"congress" + 0.029*"sign" + 0.026*"action" + 0.026*"need" + 0.022*"petit" + 0.022*"go" + 0.016*"today" + 0.015*"planet" + 0.013*"trump" + 0.012*"demand"


C:\Users\laise\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### 

C:\Users\laise\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'acuerdo'